In [ ]:
!pip install gmplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 4.0 MB/s eta 0:00:00


In [ ]:
#Cargamos las librerias que vamos a utilizar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmplot
from IPython.display import display
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

## En la celda siguiente haremos todo lo relacionado a extracción de datos



In [ ]:
#cargamos el archivo
uploaded = files.upload()

Saving datos_abiertos_2023_mayo.csv to datos_abiertos_2023_mayo.csv


In [ ]:
# Cargar el archivo CSV en un dataframe
dataframe = pd.read_csv("datos_abiertos_2023_mayo.csv")

dataframe['Inicio_del_viaje'] = pd.to_datetime(dataframe['Inicio_del_viaje'])
dataframe['Fin_del_viaje'] = pd.to_datetime(dataframe['Fin_del_viaje'])

# Calcular la duración del viaje en minutos y guardarla en la columna 'Duracion_del_viaje'
dataframe['Duracion_del_viaje'] = (dataframe['Fin_del_viaje'] - dataframe['Inicio_del_viaje']).dt.total_seconds() / 60

mapeo_genero = {'M': 0, 'F': 1} #HOMBRES (M) --> 0 Y MUJERES(F)-->1
# Aplicar el mapeo al dataframe
dataframe['Genero'] = dataframe['Genero'].map(mapeo_genero)
# Obtener el año actual
anio_actual = 2023
# Calcular la edad restando el año de nacimiento del año actual
dataframe['Edad'] = anio_actual - dataframe['Ano_de_nacimiento']
#aqui guardaremos las zonas en las que esta cada estacion
dataframe['Zona_destino'] = dataframe['Destino_Id']
# Verificar la carga exitosa imprimiendo las primeras filas del dataframe
dataframe.head()

,Viaje_Id,Usuario_Id,Genero,Ano_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Duracion_del_viaje,Edad,Zona_destino
0,28467098,70123,0.0,1967.0,2023-01-05 00:00:00,2023-01-05 00:22:00,64,141,22.0,56.0,141
1,28467099,2237235,0.0,1980.0,2023-01-05 00:00:00,2023-01-05 00:04:00,36,172,4.0,43.0,172
2,28467100,2051727,1.0,2002.0,2023-01-05 00:01:00,2023-01-05 00:10:00,96,296,9.0,21.0,296
3,28467101,2246225,0.0,1969.0,2023-01-05 00:01:00,2023-01-05 00:04:00,33,255,3.0,54.0,255
4,28467102,324247,0.0,1975.0,2023-01-05 00:01:00,2023-01-05 00:13:00,226,231,12.0,48.0,231


## Cargamos las id's, latitudes y longitudes de las estaciones

In [ ]:
uploaded = files.upload()

Saving archivo_texto (1).txt to archivo_texto (1).txt


In [ ]:
#ESTE CODIGO ES PARA DIBUJAR EL MAPA CON LOS PUNTOS

gmap = gmplot.GoogleMapPlotter(20.6597, -103.3496, 13)

latitudes = []
longitudes = []
id = []

with open("archivo_texto (1).txt", "r") as archivo:
    for linea in archivo:
        valores = linea.split(	) # Separar los valores por espacios en blanco
        id.append(int(valores[0]))
        latitudes.append(float(valores[1]))  # Convertir y guardar el valor de la columna 1
        longitudes.append(float(valores[2]))

# Obtener los datos de latitud y longitu
#  # Ejemplo de longitudes de los puntos

# Marcar los puntos en el mapa
gmap.scatter(latitudes, longitudes, color='red', size=40, marker=True)

# Guardar el mapa en un archivo HTML
gmap.draw("mapa.html")

In [ ]:
dataframe['latitudOrigen'] = dataframe['Origen_Id']
dataframe['longitudOrigen'] = dataframe['Origen_Id']
mapping1 = {}
mapping2 = {}
j = 0
for i in id:
  mapping1[i] = latitudes[j]
  mapping2[i] = longitudes[j]
  j += 1

dataframe['latitudOrigen'] = dataframe['latitudOrigen'].map(mapping1)
dataframe['longitudOrigen'] = dataframe['longitudOrigen'].map(mapping2)
dataframe = dataframe.dropna()
dataframe.head()

,Viaje_Id,Usuario_Id,Genero,Ano_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Duracion_del_viaje,Edad,Zona_destino,latitudOrigen,longitudOrigen
0,28467098,70123,0.0,1967.0,2023-01-05 00:00:00,2023-01-05 00:22:00,64,141,22.0,56.0,141,20.673072,-103.365055
1,28467099,2237235,0.0,1980.0,2023-01-05 00:00:00,2023-01-05 00:04:00,36,172,4.0,43.0,172,20.676461,-103.350244
2,28467100,2051727,1.0,2002.0,2023-01-05 00:01:00,2023-01-05 00:10:00,96,296,9.0,21.0,296,20.719530,-103.386768
3,28467101,2246225,0.0,1969.0,2023-01-05 00:01:00,2023-01-05 00:04:00,33,255,3.0,54.0,255,20.685652,-103.368215
4,28467102,324247,0.0,1975.0,2023-01-05 00:01:00,2023-01-05 00:13:00,226,231,12.0,48.0,231,20.664680,-103.333570


## Hacemos un agrupamiento por zonas de las estaciones en base a la longitud y latitud de cada una de estas

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

In [ ]:
latitudes = np.array(latitudes)[:, np.newaxis]
longitudes = np.array(longitudes)[:, np.newaxis]
X = np.concatenate([latitudes, longitudes], axis = 1)

In [ ]:
# Creamos una instancia de K-means con 6 clusters
kmeans = KMeans(n_clusters=6)

# Ajustamos el modelo K-means a los datos
kmeans.fit(X)

# Obtenemos las etiquetas de los clusters asignados a cada muestra
labels = kmeans.labels_

# Obtenemos las coordenadas de los centroides
centroids = kmeans.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Asignamos la zona de destino a la que llega cada uno de los viajes

In [ ]:
mapping = {}
j = 0
for i in id:
  mapping[i] = labels[j]
  j += 1
dataframe['Zona_destino'] = dataframe['Zona_destino'].map(mapping)
dataframe = dataframe.dropna()
dataframe.head()

,Viaje_Id,Usuario_Id,Genero,Ano_de_nacimiento,Inicio_del_viaje,Fin_del_viaje,Origen_Id,Destino_Id,Duracion_del_viaje,Edad,Zona_destino,latitudOrigen,longitudOrigen
0,28467098,70123,0.0,1967.0,2023-01-05 00:00:00,2023-01-05 00:22:00,64,141,22.0,56.0,1,20.673072,-103.365055
1,28467099,2237235,0.0,1980.0,2023-01-05 00:00:00,2023-01-05 00:04:00,36,172,4.0,43.0,5,20.676461,-103.350244
2,28467100,2051727,1.0,2002.0,2023-01-05 00:01:00,2023-01-05 00:10:00,96,296,9.0,21.0,4,20.719530,-103.386768
3,28467101,2246225,0.0,1969.0,2023-01-05 00:01:00,2023-01-05 00:04:00,33,255,3.0,54.0,3,20.685652,-103.368215
4,28467102,324247,0.0,1975.0,2023-01-05 00:01:00,2023-01-05 00:13:00,226,231,12.0,48.0,2,20.664680,-103.333570


## Calculamos la duracion de cada uno de los viajes y lo guardamos en un dataframe

In [ ]:
#Creamos una copia del dataframe
df = dataframe
#Obtenemos las distintas horas de salida de cada uno de los usuarios
df_Hora1 = df['Inicio_del_viaje'].dt.strftime('%H:%M:%S').str.slice(0, 2)
# Convertir la hora en minutos
df_HoraSalida = df_Hora1.astype(int) * 60 + df['Inicio_del_viaje'].dt.minute

#Obtenemos las distintas horas de llegada de cada uno de los usuarios
df_Hora2 = df['Fin_del_viaje'].dt.strftime('%H:%M:%S').str.slice(0, 2)
# Convertir la hora en minutos
df_HoraLlegada = df_Hora2.astype(int) * 60 + df['Fin_del_viaje'].dt.minute

def cambiar_negativos(value):
  if( value < 0): return 1440 + value

  return value

#Calculamos la duracion del recorrido de cada persona
df_Duracion = df_HoraLlegada - df_HoraSalida
df_Duracion = df_Duracion.apply(cambiar_negativos)

## Preparamos nuestros datos predictores: Edad, duracion del viaje, latitud de origen, longitud de origen y genero del usuario

In [ ]:
X_data = pd.concat([df['Edad'], df_Duracion, df['latitudOrigen'], df['longitudOrigen'], df['Genero']], axis = 1)
features = X_data.columns
X_data = X_data.loc[:, features].values
y_data = df.loc[:, ['Zona_destino']].values

## Ajustamos los modelos de Regresion logistica y LDA a nuestros datos de entrenamiento, calculamos el score con nuestros datos de prueba

In [ ]:
#separamos los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25, random_state=18)
#aplicamos regresion logistica
model = LogisticRegression()
model.fit(X_train, y_train)
# Obtener los coeficientes aprendidos
coef = model.coef_[0]
print(coef)

# Evaluar el modelo en el conjunto de prueba
score = model.score(X_test, y_test)
print("Exactitud en el conjunto de prueba:", score)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[0.00628085 0.01670153 0.03084599 0.00687921 0.05432705]
Exactitud en el conjunto de prueba: 0.42202057936095666


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
model = LinearDiscriminantAnalysis()

# Entrenar el clasificador con los datos de entrenamiento
model.fit(X_train, y_train)

coef = model.coef_[0]
print(coef)

# Calcular el puntaje de la predicción
score = model.score(X_test, y_test)

print("Puntaje de la predicción:", score)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[3.10021132e-04 8.03074617e-03 9.52002478e+01 1.70473808e+01
 7.73435856e-02]
Puntaje de la predicción: 0.5853955061395462
